<a href="https://colab.research.google.com/github/sgbaird/honegumi/blob/main/notebooks/3.0-human-in-the-loop-BO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human-in-the-loop Bayesian Optimization
(e.g., starting/stopping Python script to run wetlab experiments and collect results)

In [ ]:
%pip install ax-platform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.2/745.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from ax.service.ax_client import AxClient, ObjectiveProperties

from ax.modelbridge.factory import Models
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy

import pandas as pd

obj1_name = "branin"

# Define the training data


X_train = pd.DataFrame(
    [
        {"x1": -3.0, "x2": 5.0},
        {"x1": 0.0, "x2": 6.2},
        {"x1": 5.9, "x2": 2.0},
        {"x1": 1.5, "x2": 2.0},
        {"x1": 1.0, "x2": 9.0},
        {'x1': 3.4358130490836167, 'x2': 4.61231462984769},
        {'x1': 6.890019044982994, 'x2': 6.433171434935316},
        {'x1': 2.322108669736828, 'x2': 3.956878674674913},
    ]
)

y_train = [
    48.62023496052493,
    19.642112642270263,
    19.70361019577401,
    14.301933537630894,
    35.100744296912005,
    7.341504681144744,
    45.6214716397554,
    4.3596298892582475,
]

In [ ]:
# Define the number of training examples
n_train = len(X_train)


gs = GenerationStrategy(
    steps=[
        GenerationStep(
            model=Models.GPEI,
            num_trials=-1,
            max_parallelism=3,
        ),
    ]
)

ax_client = AxClient(generation_strategy=gs)

ax_client.create_experiment(
    parameters=[
        {"name": "x1", "type": "range", "bounds": [-5.0, 10.0]},
        {"name": "x2", "type": "range", "bounds": [0.0, 10.0]},
    ],
    objectives={
        obj1_name: ObjectiveProperties(minimize=True),
    },
)

# Add existing data to the AxClient
for i in range(n_train):
    parameterization = X_train.iloc[i].to_dict()

    ax_client.attach_trial(parameterization)
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])


parameterization, trial_index = ax_client.get_next_trial()

best_parameters, metrics = ax_client.get_best_parameters()
print(best_parameters)

print(f"Next suggested experiment: ", parameterization)

[INFO 03-11 22:36:27] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-11 22:36:27] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-11 22:36:27] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-11 22:36:27] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-5.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 10.0])], parameter_constraints=[]).
[INFO 03-11 22:3

{'x1': 2.322108669736828, 'x2': 3.956878674674913}
Next suggested experiment:  {'x1': 10.0, 'x2': 0.0}


As a simple way to run "human-in-the-loop" Bayesian optimization, whenever you get a new suggested experiment and carry out the experiment, manually add it to `X_train` and `y_train` defined previously and run the code again.